<a href="https://colab.research.google.com/github/AlexGitta/DAID/blob/main/prototype3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MuJoCo imitation prototype

For this prototype I decided I wanted to focus on creating an interactive and easy to use human imitation program in MuJoCo. Therefore, I did not attempt to implement custom pose imitation, instead I used the keyframes included in the humanoid model by DeepMind. My future plan is to create a pipeline where users can perform poses or motions that the agent will then try to imitate.

## Installs and Imports

In [ ]:
# Install packages using pip
!pip install mujoco
!pip install -q mediapy

In [ ]:
# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')
else:
  print('GPU connected.')

%env MUJOCO_GL=egl


GPU connected.
env: MUJOCO_GL=egl


In [ ]:
# Import packages
try:
  import mujoco
  import mediapy as media
  import numpy as np
  from typing import Callable, Optional, Union, List
  import scipy.linalg
  import matplotlib.pyplot as plt
  from IPython.display import clear_output
except Exception as e:
   raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above ')

print('Installed packages')

Installed packages


In [ ]:
# Clone humanoid from github
!git clone https://github.com/google-deepmind/mujoco
with open('mujoco/model/humanoid/humanoid.xml', 'r') as f:
  humanoid = f.read()


## Set up and render humanoid


Here I load the humanoid into the model and data variables, then render and create a video where the model is instantiated into the specified pose. Feel free to change the specified pose, and see how the model falls/ragdolls due to no joint actuators being used.

In [ ]:
model = mujoco.MjModel.from_xml_string(humanoid)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

DURATION  = 3   # seconds
FRAMERATE = 60  # Hz

# 0 for squat pose, 1 for balancing pose
mujoco.mj_resetDataKeyframe(model, data, 0)

frames = []
while data.time < DURATION:
  mujoco.mj_step(model, data)

  # render and save frames
  if len(frames) < data.time * FRAMERATE:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

# use frames to create and display video
media.show_video(frames, fps=FRAMERATE)

This browser does not support the video tag.


The problem we are now presented with is operating the joint actuators at a minimum cost. We have a desired acceleration (zero, as we are attempting to balance), and we have to calculate the required forces on each joint to achieve this. To do this, we will use SciPy to create a Linear-Quadratic regulator, as well as inverse dynamics to work out the optimal control and minimise the cost function.

## Inverse dynamics
We will set the desired position to our pose, set the desired acceleration to zero and then use the inverse dynamics function in MuJoCo to take the desired acceleration and compute the forces required to create it. Here we print the forces found by the inverse dynamics for the one leg balance pose:

In [ ]:
mujoco.mj_resetDataKeyframe(model, data, 1)
mujoco.mj_forward(model, data)  # step forward once
data.qacc = 0  # desired acceleration is zero
mujoco.mj_inverse(model, data)
print(data.qfrc_inverse)

[ 0.00000000e+00  0.00000000e+00  2.75878566e+02 -3.31855341e+01
  4.99547913e+00 -6.68771278e+00 -4.30520866e+00  3.69324103e+00
 -1.54511509e+01 -1.09062996e+01  4.11835550e-01 -1.61300544e+00
 -9.79255144e+00 -2.31182930e+00 -3.66106442e-01 -5.91255024e+00
 -4.17112597e-01 -1.91408228e+00  5.75890779e+00  2.66518759e+00
 -2.02066315e-01 -5.75454992e+00  9.94329230e-01  1.14109244e+00
 -1.98740242e+00  3.82112740e+00  1.15105404e+00]


The largest force is clearly the 3rd in the list, the vertical aspect of the root joint. This is true for both the one leg and the squat pose (Try it yourself! Just change the one for a zero). This is not natural, as we as humans cannot just will our bodies upward to the sky, as if the rapture has come. To fix this, we have to offset the humanoid a little bit from the ground.

### Finding the optimal offset from the ground


This function tries 2001 different offsets and calculates the optimal one where the vertical forces are minimal.

In [ ]:
def findOffset(squatorbalance):

    height_offsets = np.linspace(-0.001, 0.001, 2001) # create array of 2001 potential optimal offsets
    vertical_forces = []
    for offset in height_offsets: # loop through array
      mujoco.mj_resetDataKeyframe(model, data, squatorbalance)
      mujoco.mj_forward(model, data) # step forward once
      data.qacc = 0
      data.qpos[2] += offset  # offset the height
      mujoco.mj_inverse(model, data)   # use inverse dynamics to calculate required forces
      vertical_forces.append(data.qfrc_inverse[2])  # add the force to the list

    # find the offset at which the force is smallest.
    idx = np.argmin(np.abs(vertical_forces))
    best_offset = height_offsets[idx]
    return best_offset


print(findOffset(0))

-0.000353


Using this function in our original force calculation results in alot cleaner forces.

In [ ]:
def calculateForces(squatorbalance):
  mujoco.mj_resetDataKeyframe(model, data, squatorbalance)
  mujoco.mj_forward(model, data)
  data.qacc = 0
  data.qpos[2] += findOffset(squatorbalance)
  qpos0 = data.qpos.copy()
  mujoco.mj_inverse(model, data)
  qfrc0 = data.qfrc_inverse.copy()
  return qfrc0, qpos0

As you can hopefully see, these forces are a lot more regular. Now all we have to do is use MuJoCo's genius to calculate the actuator values that will create these forces, and then save them into a

## Calculating our first actuator values

In [ ]:
def calculateActuatorValues(squatorbalance):
  qfrc0, qpos0 = calculateForces(squatorbalance)
  actuator_moment = np.zeros((model.nu, model.nv))
  mujoco.mju_sparse2dense(
      actuator_moment,
      data.actuator_moment.reshape(-1),
      data.moment_rownnz,
      data.moment_rowadr,
      data.moment_colind.reshape(-1),
  )
  ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(actuator_moment)
  ctrl0 = ctrl0.flatten()
  data.ctrl = ctrl0
  return ctrl0, qpos0

Now we can simulate the agent with their new knowledge of the first actuator values.

In [ ]:
ctrl0, qpos0 = calculateActuatorValues(1) # change to choose squat or one leg balance

DURATION  = 3   # seconds
FRAMERATE = 60  # Hz
mujoco.mj_resetData(model, data)
data.qpos = qpos0
data.ctrl = ctrl0

frames = []
while data.time < DURATION:
  mujoco.mj_step(model, data)

      # render and save frames
  if len(frames) < data.time * FRAMERATE:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

This browser does not support the video tag.


Here we can see for both pose attempts, the humanoid still falls, but it does attempt to stabilize itself and fight against gravity for a moment. Next, we need to allow it to feedback to itself, so it can update the actuator forces as the simulation happens. First, we set up a cost for balancing that will aim to keep the centre of mass over the foot (or feet if squatting). We will use kinematic jacobians and use MuJoCo to compute them because it is GENIUS.

In [ ]:
def balancingCost(squatorbalance):
    nu = model.nu  # alias for the number of actuators
    R = np.eye(nu)
    nv = model.nv  # shortcut for the degrees of freedom

    # Get the Jacobian for the root body (torso) CoM
    mujoco.mj_resetData(model, data)
    data.qpos = qpos0
    mujoco.mj_forward(model, data)
    jac_com = np.zeros((3, nv))
    mujoco.mj_jacSubtreeCom(model, data, jac_com, model.body('torso').id)

    if(squatorbalance == 1):
      jac_foot = np.zeros((3, nv))
      mujoco.mj_jacBodyCom(model, data, jac_foot, None, model.body('foot_left').id)

    if(squatorbalance == 0):
      jac_foot_left = np.zeros((3, nv))
      mujoco.mj_jacBodyCom(model, data, jac_foot_left, None, model.body('foot_left').id)
      jac_foot_right = np.zeros((3, nv))
      mujoco.mj_jacBodyCom(model, data, jac_foot_right, None, model.body('foot_right').id)
      jac_foot = (jac_foot_left + jac_foot_right)/2


    jac_diff = jac_com - jac_foot
    Qbalance = jac_diff.T @ jac_diff
    return Qbalance

We need another cost function to ensure the humanoid maintains the pose, or atleast as close to it as possible. The most important joints, e.g. the legs and abdominals, need the strictest cost function, while the less crucial joints, such as the arms, are less strict, allowing the humanoid to move itself to remain balanced, as a human does.

In [ ]:
def poseCost(squatorbalance):
    nv = model.nv  # shortcut for the degrees of freedom
    # Get all joint names.
    joint_names = [model.joint(i).name for i in range(model.njnt)]

    # Get indices into relevant sets of joints.
    root_dofs = range(6)
    body_dofs = range(6, nv)
    abdomen_dofs = [
        model.joint(name).dofadr[0]
        for name in joint_names
        if 'abdomen' in name
        and not 'z' in name
    ]
    left_leg_dofs = [
        model.joint(name).dofadr[0]
        for name in joint_names
        if 'left' in name
        and ('hip' in name or 'knee' in name or 'ankle' in name)
        and not 'z' in name
    ]
    if squatorbalance == 0:
        right_leg_dofs = [
            model.joint(name).dofadr[0]
            for name in joint_names
            if 'right' in name
            and ('hip' in name or 'knee' in name or 'ankle' in name)
            and not 'z' in name
        ]
        balance_dofs = abdomen_dofs + left_leg_dofs + right_leg_dofs
    else:
        balance_dofs = abdomen_dofs + left_leg_dofs

    other_dofs = np.setdiff1d(body_dofs, balance_dofs)
    return balance_dofs, other_dofs, root_dofs

In [ ]:
def constructQ(squatorbalance):
    nv = model.nv  # shortcut for the degrees of freedom
    # get indices into relevant sets of joints
    balance_dofs, other_dofs, root_dofs = poseCost(squatorbalance)
    # Cost coefficients.
    BALANCE_COST        = 1000  # Balancing.
    BALANCE_JOINT_COST  = 3     # Joints required for balancing.
    OTHER_JOINT_COST    = .3    # Other joints.

    # Construct the Qjoint matrix.
    Qjoint = np.eye(nv)
    Qjoint[root_dofs, root_dofs] *= 0  # Don't penalize free joint directly.
    Qjoint[balance_dofs, balance_dofs] *= BALANCE_JOINT_COST
    Qjoint[other_dofs, other_dofs] *= OTHER_JOINT_COST

    # Construct the Q matrix for position DoFs.
    Qpos = BALANCE_COST * balancingCost(squatorbalance) + Qjoint

    # No explicit penalty for velocities.
    Q = np.block([[Qpos, np.zeros((nv, nv))],
                  [np.zeros((nv, 2*nv))]])
    return Q

In [ ]:
def constructK(squatorbalance):
    nu = model.nu
    nv = model.nv
    ctrl0, qpos0 = calculateActuatorValues(squatorbalance)

    # Set the initial state and control.
    mujoco.mj_resetData(model, data)
    data.ctrl = ctrl0
    data.qpos = qpos0

    # Allocate the A and B matrices, compute them.
    A = np.zeros((2*nv, 2*nv))
    B = np.zeros((2*nv, nu))
    epsilon = 1e-6
    flg_centered = True
    mujoco.mjd_transitionFD(model, data, epsilon, flg_centered, A, B, None, None)#
    return A, B

In [ ]:
squatorbalance = 0

A, B = constructK(squatorbalance)
Q = constructQ(squatorbalance)
R = np.eye(model.nu)
ctrl0, qpos0 = calculateActuatorValues(squatorbalance)


# Solve discrete Riccati equation.
P = scipy.linalg.solve_discrete_are(A, B, Q, R)

# Compute the feedback gain matrix K.
K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

# Parameters.
DURATION = 5          # seconds
FRAMERATE = 60        # Hz

# Reset data, set initial pose.
mujoco.mj_resetData(model, data)
data.qpos = qpos0

# Allocate position difference dq.
dq = np.zeros(model.nv)

frames = []
while data.time < DURATION:
  # Get state difference dx.
  mujoco.mj_differentiatePos(model, dq, squatorbalance, qpos0, data.qpos)
  dx = np.hstack((dq, data.qvel)).T

  # LQR control law.
  data.ctrl = ctrl0 - K @ dx

  # Step the simulation.
  mujoco.mj_step(model, data)

  # Render and save frames.
  if len(frames) < data.time * FRAMERATE:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

<ipython-input-95-fbef6b093e59>:33: RuntimeWarning: invalid value encountered in matmul
  data.ctrl = ctrl0 - K @ dx


This browser does not support the video tag.
